In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
# import folium
# from folium.plugins import HeatMap
from scipy import stats
from ipyleaflet import Map, Marker, CircleMarker
from ipywidgets import SelectionSlider, VBox, interact
import pandas as pd

# Loading the datasets
sales_data = pd.read_csv('data/sales_data.csv')
customer_data = pd.read_csv('data/customer_data.csv')
shopping_mall_data = pd.read_csv('data/shopping_mall_data.csv')

In [ ]:
# Exploratory display of the sales data
sales_data["invoice date"] = pd.to_datetime(sales_data["invoice date"])
sale_date = sales_data.groupby(["shopping_mall", "invoice date"], as_index=False)["price"].sum()
sale_with_coord = pd.merge(sale_date, shopping_mall_data, how='left', on='shopping_mall')[['shopping_mall', 'invoice date', 'price', 'lon', 'lat']]

# 提取所有日期
dates = sorted(sale_with_coord["invoice date"].dt.strftime("%Y-%m-%d").unique())

# 获取每个x的经纬度
locations = sale_with_coord.groupby('shopping_mall')[["lat", "lon"]].first().reset_index()

# 创建地图
m = Map(center=(36.7783, -119.4179), zoom=6)

# 创建滑动条
slider = SelectionSlider(
    options=dates,
    value=dates[0],
    description="Date:",
    continuous_update=True
)

# 创建动态更新函数
circles = {}  # 用于存储地图上的 CircleMarker 对象
popups = {}

def update_map(selected_date):
    # 清理旧的 CircleMarker
    for circle in circles.values():
        #m.remove_layer(circle)
        circle.radius = 0
    #circles.clear()
    #for popup in popups.values():
        #m.remove_layer(popup)
    #popups.clear()
    
    # 获取选中日期的汇总数据
    current_data = sale_with_coord[sale_with_coord["invoice date"] == pd.to_datetime(selected_date)]
    for _, row in current_data.iterrows():
        x = row["shopping_mall"]
        z_value = row["price"]
        loc = locations[locations["shopping_mall"] == x]
        latitude = loc["lat"].values[0]
        longitude = loc["lon"].values[0] 

        if x in circles:
            circle = circles[x]
        else:
        # 创建 CircleMarker
            circle = CircleMarker(
                location=(latitude, longitude),
                radius=int(z_value / 1000),  # 根据z值调整圆的大小
                color="blue",
                fill_color="blue",
                fill_opacity=0.7
            )
            circles[x] = circle
            m.add_layer(circle)

    for _, row in current_data.iterrows():
        x = row["shopping_mall"]
        loc = locations[locations["shopping_mall"] == x]
        latitude = loc["lat"].values[0]
        longitude = loc["lon"].values[0]
        circle = circles[x]

        if x in popups:
            print(x)
            #m.remove_layer(popup)
            popup = popups[x]
            popup.child = HTML(f"<b>{row['shopping_mall']}</b><br>Total Sales: {row['price']}<br>Scaled:")
            m.add_layer(popup)
        else:
            popup_content = HTML(f"<b>{row['shopping_mall']}</b><br>Total Sales: {row['price']}<br>Scaled:")
            popup = Popup(location=(latitude, longitude), child=popup_content, close_button=True)        
            popups[x] = popup
            circle.popup = popup
            m.add_layer(popup)


        

# 滑动条事件绑定
def on_slider_change(change):
    update_map(change["new"])

slider.observe(on_slider_change, names="value")

# 显示初始地图和滑动条
update_map(dates[0])  # 初始化地图

# interact(lambda index: update_map(index), index=slider)
VBox([slider, m])


Map(center=[36.7783, -119.4179], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', '…